# Задание 4

In [1]:
!pip install -U trl bitsandbytes datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 10.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 35.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 30.0 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.4.1
    Uninstalling datasets-4.4.1:
      Successfully uninstalled datasets-4.4.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0
    Uninstalling accelerate-1.11.0:
      Successfully uninstalled accelerate-1.11.0


Остановился на датасете igorktech/anekdots, где собраны анекдоты посредственного качества за 30 лет с форума. Перед подачей датасета в модель, я выполнил фильтрацию от нецензурной лексики (поиск подстроки - корня нецензурного слова, сам список корней был удален из ноутбука из этических соображений), а также попробовал откидывать слишком длинные шутки. Например, пробовал откидывать анекдоты длиннее 200 символов, но тогда качество генерируемых анекдотов заметно ухудшалось, что можно будет увидеть в самом конце ноутбука. лучше всего подошло ограничение в 500 символов (пробовал 150, 200, 300, 500)

In [3]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments

dataset = load_dataset("igorktech/anekdots", split="train")

import re

BAD_ROOTS = []

def contains_bad_words(text):
    if not text: return False
    text_lower = text.lower()
    for root in BAD_ROOTS:
        if root in text_lower:
            return True
    return False


print(f"До фильтрации: {len(dataset)}")
dataset = dataset.filter(lambda x: not contains_bad_words(str(x['text'])))
print(f"После фильтрации нецензурных слов: {len(dataset)}")
dataset = dataset.filter(lambda x: x['text'] and 10 < len(str(x['text'])) < 200)
print(f"После фильтрации длины: {len(dataset)}")
dataset = dataset.shuffle(seed=42).select(range(min(50000, len(dataset))))
print(f"Итоговый размер для обучения: {len(dataset)}")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/122M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/497596 [00:00<?, ? examples/s]

До фильтрации: 497596


Filter:   0%|          | 0/497596 [00:00<?, ? examples/s]

...

После фильтрации длины: 164094
Итоговый размер для обучения: 50000


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Из предложенных моделей выбрал самую новую и ту, которая имеет наибольшее число параметров, а именно Qwen/Qwen3-0.6B

In [ ]:
model_id = "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

Задал простой системный промпт, форсирующий модель цензурировать свою речь, а также заканчивать свои шутку панчлайном. Модель, кстати, иногда воспринимала эту просьбу буквально и генерировала что-то вроде "...панчлайн".

С вероятностью в 50% в instruct может пойти часть шутки из датасета (затравка), либо обычный "юзерский" промпт. Надеемся, что тогда модель в целом поймет, что такое анекдот

In [ ]:
import random

def formatting_prompts_func(examples):
    SYSTEM_PROMPT = (
        "Ты — тактичный стендап-комик. Твоя задача — рассказывать смешные анекдоты на русском языке. "
        "Правила: 1. Никакого мата (используй цензурные аналоги). "
        "2. Никаких оскорблений. 3. Всегда заканчивай шутку панчлайном."
    )

    GENERIC_PROMPTS = [
        "Расскажи анекдот.",
        "Нужна шутка без мата.",
        "Рассмеши меня анекдотом."
    ]

    output_texts = []
    
    for text in examples['text']:
        text = str(text).strip()
        words = text.split()
        
        mode = "full"
        split_index = 0
        
        # 50% шанс симуляции затравки
        if len(words) > 10 and random.random() < 0.5:
            mode = "continuation"
            split_index = random.randint(3, min(8, len(words) - 5))
            
        messages = [{"role": "system", "content": SYSTEM_PROMPT}]

        if mode == "continuation":
            prefix = " ".join(words[:split_index])
            suffix = " ".join(words[split_index:])
            user_msg = f'Продолжи анекдот, вот начало: "{prefix}"'
            assistant_msg = suffix
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        else:
            user_msg = random.choice(GENERIC_PROMPTS)
            assistant_msg = text
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})

        # генерируем текст
        formatted_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        # в конце добавляю eos
        formatted_text += tokenizer.eos_token
        
        output_texts.append(formatted_text)

    return {"text": output_texts}

formatted_dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=dataset.column_names
)

In [5]:
print(formatting_prompts_func(dataset[:1]))

{'text': ['<|im_start|>system\nТы — тактичный стендап-комик. Твоя задача — рассказывать смешные анекдоты на русском языке. Правила: 1. Никакого мата (используй цензурные аналоги). 2. Никаких оскорблений. 3. Всегда заканчивай шутку панчлайном.<|im_end|>\n<|im_start|>user\nПродолжи анекдот, вот начало: "День рождение - это один день в..."<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\nгоду, все остальные дни - это его празднование.<|im_end|>\n<|im_end|>']}


Использую LORA адаптеры, иначе не хватало памяти

In [6]:
from peft import LoraConfig, TaskType, get_peft_model

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    # все линейные слои
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Перепробовал разные гиперпараметры, но заметил, что в случае, если лосс падает до 0.8 и ниже, то модель хорошо выучивает структуру сообщения, т.е. действительно пытается продолжить анекдот по затравке, однако при этом переобучается и начинает генерировать заученные фразы из датасета совершенно не к месту.

In [7]:
from trl import SFTTrainer, SFTConfig

training_args = SFTConfig(
    output_dir="./qwen3-0.6b-anekdot-lora",
    peft_config=peft_config
    num_train_epochs=1,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    dataloader_num_workers=4,
    dataloader_pin_memory=True,
    learning_rate=4e-6,
    fp16=True if torch.cuda.is_available() else False,
    logging_steps=10,
    save_strategy="epoch",
    dataset_text_field="text",
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    peft_config=peft_config,
    args=training_args,
    processing_class=tokenizer
)

trainer.train()
trainer.save_model("./final_checkpoint")

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Adding EOS to train dataset:   0%|          | 0/50000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/50000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/50000 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Начинаем обучение...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Step,Training Loss
10,3.569800
20,3.289800
30,3.045800
40,2.821400
50,2.611300
60,2.426300
70,2.242000
80,2.095300
90,1.913800
100,1.734400


В этом запуске можно было остановить на ~500 эпохе, а не ждать до 1500

Для генерации подошла температура в промежутке 0.6-0.7, если ставил выше, то модель начинала генерировать еще больше бреда, а если ниже - повторяла заученные анекдоты. Пришлось выставить repetition_penalty=1.15, чтобы модель не начинала повторять одну и ту же фразу. Очень важными параметрами оказались min_new_tokens и max_new_tokens. Если не ограничить по ним модель, то генерации становятся слишком короткими, либо при достижении определенной длины полностью теряют смысл. В целом именно параметры генерации и предобработка данных оказали наибольший эффект на качество генерации

In [28]:
import os
from tqdm import tqdm
from transformers import AutoTokenizer

input_prompts_file = "/kaggle/input/prefixes/prefixes.txt"
output_jokes_file = "submission.txt"
model_id = "Qwen/Qwen3-0.6B"

print("Загрузка токенизатора...")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

SYSTEM_PROMPT = (
    "Ты — тактичный стендап-комик. Твоя задача — рассказывать смешные анекдоты на русском языке. "
    "Правила: 1. Никакого мата (используй цензурные аналоги). "
    "2. Никаких оскорблений. 3. Всегда заканчивай шутку панчлайном."
)

lines = []
if os.path.exists(input_prompts_file):
    with open(input_prompts_file, "r", encoding="utf-8") as f:
        lines = f.readlines()

generated_entries = []
model.eval()

print(f"Обработка {len(lines)} промптов...")

for line in tqdm(lines):
    line = line.strip()
    if not line: continue
    
    match = re.match(r"^(\d+)[\.\)]?\s+(.*)", line)
    if not match: continue
    
    prompt_id = match.group(1)
    prompt_text = match.group(2)
    
    user_content = f'Продолжи анекдот, вот начало: "{prompt_text}"'
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_content}
    ]

    full_prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True 
    )
    
    inputs = tokenizer([full_prompt], return_tensors="pt").to(model.device)
    
    # генерация
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            min_new_tokens=40,
            do_sample=True,
            temperature=0.65,
            top_k=50,
            top_p=0.95, 
            repetition_penalty=1.15,
            pad_token_id=tokenizer.eos_token_id
        )
    
    input_len = inputs.input_ids.shape[1]
    generated_ids = outputs[0][input_len:]
    generated_suffix = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
    
    # склеиваю затравку с генерацией
    if generated_suffix.lower().startswith(prompt_text.lower()[:15]):
        full_joke = generated_suffix 
    else:
        full_joke = f"{prompt_text} {generated_suffix}"

    # добавил на всякий случай
    full_joke = full_joke.replace("\n", " ").replace("  ", " ").strip()
    generated_entries.append(f"{prompt_id} {full_joke}")

with open(output_jokes_file, "w", encoding="utf-8") as f:
    for entry in generated_entries:
        f.write(entry + "\n")
        print(entry)
print(f"Готово. Сохранено в {output_jokes_file}")

Загрузка токенизатора...
Обработка 77 промптов...


100%|██████████| 77/77 [05:35<00:00,  4.36s/it]

1 Идёт мужик по лесу застряли в деревне. Следователи не могли идти." переполненность земляных криколов как садит багрянку в корону!
2 Встречаются два друга - Дружеская встреча? Отлично!" Это не всегда счастливый момент." — Но если они не хотят встречаться?" "Нет! Не бойтесь!"
3 Приходят мужик в бар сидя за кружкой и хлебом, думаю..." "Вы не знаете, что значит быть незаведенным?" - спросил один из них.
4 Жена говорит мужу - Ладно, каким вы будете работать? Соблюдайте срочность!" Мужу не нужно срочной работы! Думаю, что в нашем случае все срочно!
5 Приходят альфа, бета и гамма в бар - Они откажутся от продажи... по сравнению с нами." "Если бы они все перестали давать деньги, то не было бы ни одного трейсера!"
6 Идёт медведь по лесу - Идёт медведь по лесу... и не сидит подруге." "Нет. Он поймал его в тучу!" 🔸
7 Приходит мужик к врачу и говорит ему: - Простите вам, но мне нужно подумать...". история с глаголом "подумать" не всегда начинается на суждении.
8 Встречаются русский, американец и

Это не самый лучший ран, но у меня закончилось время под gpu на kaggle и colab :(